# Rewrite 를 하는 RAG 만들기

- 답변이 넣어준 정보와 관련이 없다면, 질문을 바꿔서 다시 질의하는 flow 를 추가해보기
- 참고자료: [공식문서](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/)

In [1]:
import uuid

from IPython.display import HTML
from langchain_core.runnables.graph import Graph


def draw_mermaid_with_html(g: Graph):
    mermaid_code = g.draw_mermaid()

    # 고유 ID 생성
    container_id = f"mermaid-container-{uuid.uuid4().hex[:8]}"

    html = f"""
    <div id="{container_id}"></div>

    <script type="module">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';

        mermaid.initialize({{
            startOnLoad: false,
            theme: 'default',
            securityLevel: 'loose'
        }});

        const graphDefinition = `{mermaid_code}`;
        const container = document.getElementById('{container_id}');

        try {{
            const {{ svg }} = await mermaid.render('graphDiv-' + Date.now(), graphDefinition);
            container.innerHTML = svg;
        }} catch (error) {{
            container.innerHTML = '<pre>' + error + '</pre>';
        }}
    </script>
    """

    display(HTML(html))

## 1. Retriever 및 llm 정의

In [2]:
from pathlib import Path

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings


DATA_DIR = Path("../data")

chroma_persistent_dir = DATA_DIR / "chroma_db"

embedding_function = OllamaEmbeddings(model="bge-m3:567m")
vector_store = Chroma(
    embedding_function=embedding_function,
    persist_directory=chroma_persistent_dir,
    collection_name="korean_income_tax_law",
)

In [3]:
income_tax_law_retriever = vector_store.as_retriever(kwars={"k": 1})

In [4]:
from langchain_ollama import ChatOllama


llm = ChatOllama(model="gpt-oss:20b")

## 2. Graph Builder 만들기

### 2.1. State 만들기

In [5]:
from typing import Literal, TypedDict

from langchain_core.documents import Document


class AgentState(TypedDict):
    query: str
    context: list[Document]
    answer: str

## 2.2 builder 만들기

In [6]:
from langgraph.graph import StateGraph


graph_builder = StateGraph(AgentState)

## 3. Node 만들기

### 3.1. 검색과 생성 함수 만들기

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


def retrieve(state: AgentState) -> AgentState:
    query = state["query"]
    retrieved_docs = income_tax_law_retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    return {"context": context}


def generate(state: AgentState) -> AgentState:
    query = state["query"]
    context = state["context"]

    prompt = PromptTemplate.from_template("""역할: 당신은 한국의 소득세법 전문가입니다.
---
지시: 다음 정보를 바탕으로 사용자의 질문에 답하세요.
---
정보: {context}
---
사용자 질문: {query}""")

    rag_chain = prompt | llm | StrOutputParser()

    answer = rag_chain.invoke(
        {
            "query": query,
            "context": context,
        }
    )
    return {"answer": answer}

### 3.2. 관련성 판별기와 질문 재생성 함수 만들기

In [8]:
# conditional edge 로 만들 것이기 때문에 Node 이름은 Literal 로 넘겨야함
def determine_doc_relevance(state: AgentState) -> Literal["rewrite", "generate"]:
    query = state["query"]
    context = state["context"]
    prompt = PromptTemplate.from_template("""역할: '검색된 문서 내용'과 '사용자의 질문'이 얼마나 연관이 있는지 등급 매기기입니다.
---
지시: 문서가 사용자 질문과 키워드나 의미적으로 관련이 있으면 1, 없으면 0을 반환하세요. 엄격한 기준이 아닌, 명백히 무관한 문서만 필터링하는 것이 목적입니다.
---
출력: 관련이 있으면 1, 없으면 0
---
검색된 문서 내용:
{documents}
---
사용자 질문: {question}
""")
    doc_relevance_chain = prompt | llm | StrOutputParser()
    is_relevant = doc_relevance_chain.invoke({"question": query, "documents": context})
    print("    ", "[determine_doc_relevance]", "is_relevant:", type(is_relevant), is_relevant)
    return "generate" if is_relevant == "1" else "rewrite"

In [9]:
def rewrite(state: AgentState) -> AgentState:
    original_query = state["query"]
    dictionary = ["- 사람과 관련된 표현 -> 거주자"]
    prompt = PromptTemplate.from_template(f"""역할: 당신은 질문 변환기입니다.
---
지시: 사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
---
출력: 변경된 질문
---
사전:
{dictionary}
---
질문: {{query}}
""")
    rewrite_query_chain = prompt | llm | StrOutputParser()
    query = rewrite_query_chain.invoke(original_query)
    print("    ", "[rewrite]", original_query, "->", query)
    return {"query": query}

## 4. Node 및 Edge 추가하기

In [10]:
graph_builder.add_sequence([retrieve, generate])
graph_builder.add_node("rewrite", rewrite)
# conditional edge 를 넘기는 node 인 determine_doc_relevance 는 Node 로 추가하지 않음

In [11]:
from langgraph.graph import END, START


graph_builder.add_edge(START, "retrieve")
graph_builder.add_conditional_edges("retrieve", determine_doc_relevance)
graph_builder.add_edge("rewrite", "retrieve")
graph_builder.add_edge("generate", END)

## 5. Graph 생성

In [12]:
basic_agent_graph = graph_builder.compile()
draw_mermaid_with_html(basic_agent_graph.get_graph())

In [13]:
basic_agent_graph.invoke({"query": "연봉 5천 만원인 사람의 소득세는?"})

     [determine_doc_relevance] is_relevant: <class 'str'> 1


{'query': '연봉 5천 만원인 사람의 소득세는?',
 'context': '과세기간의 총수입금액에서 이에 사용된 필요경비를 공제한 금액으로 하며, 필요경비가 총 수입금액을 초과하는 경우 그 초과하는 금액을 "결손금"이라 한다. ③ 제1항 각 호에 따른 사업의 범위에 관하여는 이 법에 특별한 규정이 있는 경우 외에는 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준산업분류에 따르고, 그 밖의 사업소득의 범위에 관하여 필요한 사항은 대통령령으로 정한다.# [전문개정 2009. 12. 31.]제20조(근로소득) ① 근로소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2016. 12. 20., 2024. 12. 31.>- 1. 근로를 제공함으로써 받는 봉급 · 급료 · 보수 · 세비 · 임금 · 상여 · 수당과 이와 유사한 성질의 급여 - 2. 법인의 주주총회 · 사원총회 또는 이에 준하는 의결기관의 결의에 따라 상여로 받는 소득 - 3. 「법인세법」에 따라 상여로 처분된 금액 - 4. 퇴직함으로써 받는 소득으로서 퇴직소득에 속하지 아니하는 소득 - 5. 종업원등 또는 대학의 교직원이 지급받는 직무발명보상금(제21조제1항제22호의2에 따른 직무발명보상금은 제 - 외한다) - 6. 사업자나 법인이 생산 · 공급하는 재화 또는 용역을 그 사업자나 법인(「독점규제 및 공정거래에 관한 법률」 에 따 - 른 계열회사를 포함한다)의 사업장에 종사하는 임원등에게 대통령령으로 정하는 바에 따라 시가보다 낮은 가격 - 으로 제공하거나 구입할 수 있도록 지원함으로써 해당 임원등이 얻는 이익 - ② 근로소득금액은 제1항 각 호의 소득의 금액의 합계액(비과세소득의 금액은 제외하며, 이하 "총급여액"이라 한다 - )에서 제47조에 따른 근로소득공제를 적용한 금액으로 한다. - ③ 근로소득의 범위에 관하여 필요한 사항은 대통령령으로 정한다. - [전문개정 2009. 12. 31.]\n\n지급) 국세청장은 제150조에 따라 근로소득에 대한 소득세를 징수하여 납부